<a href="https://colab.research.google.com/github/DmitriMAI/to_future/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22GPT_introduction_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [3]:
from datasets import load_dataset, Dataset

In [4]:
dataset = load_dataset("text", data_files={"train": "./train_dataset.txt"}, sample_by="document")

Using custom data configuration default-974fcbb212fe5ba9
Found cached dataset text (C:/Users/Dima/.cache/huggingface/datasets/text/default-974fcbb212fe5ba9/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
joke_datasets = []

In [6]:
k = 0
for index, item in enumerate(dataset['train'][0]['text'].split('\n\n\n\n')):
    k = k +1
    if (k > 50): #ВОт тут длинна датасета !!!!!
        break
    joke_datasets.append({"text":item})

In [7]:
dataset = Dataset.from_list(joke_datasets, split="train+test")

In [8]:
dataset[0]

{'text': '- Как водичка ?\n- А я здесь как женшина сижу, а не как термометр.'}

In [9]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"]))

  0%|          | 0/50 [00:00<?, ?ex/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuner",
    learning_rate=2e-3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
        optimizers = (torch.optim.AdamW(model.parameters(),lr=2e-3),None) # Optimizer and lr scheduler
)

In [13]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Assigning [PAD] to the pad_token key of the tokenizer
Adding [PAD] to the vocabulary


1

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 50
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 250


  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
text = "В дверь постучали"
input_ids = tokenizer.encode(text, return_tensors="pt").to("cuda:0")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В дверь постучали.
- Войдите, доктор.
- Что вам нужно?
- Принесите стакан воды.
- Нет, я хочу пить.
- Возьмите.
- Нет, я хочу есть.
- Что вам нужно?
- Возьмите мой носовой платок.
- Что вам нужно, чтобы вытирать пот со лба?
- Простите, доктор, но у меня болит голова.
- Скажите, доктор, у вас есть носовой платок?
- Есть, доктор
